# Random Acts of Pizza - Baseline Model

This notebook creates a baseline model for predicting which pizza requests will be successful.

## Approach
- Load JSON data
- Extract basic numerical features
- Simple text features (length, word count)
- LightGBM classifier
- 5-fold cross-validation

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Load data
print("Loading training data...")
with open('/home/data/train.json', 'r') as f:
    train_data = json.load(f)

print(f"Training samples: {len(train_data)}")
print(f"Sample keys: {list(train_data[0].keys())}")

Loading training data...
Training samples: 2878
Sample keys: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_request', 're

In [3]:
# Feature engineering - basic numerical features
print("Creating basic features...")

# Numerical features that should be available at prediction time
numerical_features = [
    'requester_account_age_in_days_at_request',
    'requester_number_of_comments_at_request',
    'requester_number_of_posts_at_request',
    'requester_number_of_subreddits_at_request',
    'requester_upvotes_minus_downvotes_at_request',
    'requester_upvotes_plus_downvotes_at_request',
    'requester_number_of_comments_in_raop_at_request',
    'requester_number_of_posts_on_raop_at_request',
    'requester_days_since_first_post_on_raop_at_request'
]

# Create feature DataFrame
features_df = train_df[numerical_features].copy()

# Fill missing values with median
for col in numerical_features:
    if features_df[col].isnull().any():
        median_val = features_df[col].median()
        features_df[col].fillna(median_val, inplace=True)
        print(f"Filled missing values in {col} with median: {median_val:.2f}")

# Text-based features - use request_text_edit_aware which is available in both train and test
print("\nCreating text features...")
features_df['title_length'] = train_df['request_title'].fillna('').str.len()
features_df['text_length'] = train_df['request_text_edit_aware'].fillna('').str.len()
features_df['title_word_count'] = train_df['request_title'].fillna('').str.split().str.len()
features_df['text_word_count'] = train_df['request_text_edit_aware'].fillna('').str.split().str.len()

# User flair feature
print("Creating user flair feature...")
features_df['has_flair'] = (train_df['requester_user_flair'].notna() & 
                           (train_df['requester_user_flair'] != 'None') &
                           (train_df['requester_user_flair'] != None)).astype(int)

# Post was edited
features_df['post_was_edited'] = train_df['post_was_edited'].astype(int)

print(f"Final feature shape: {features_df.shape}")
print(f"Features: {list(features_df.columns)}")

Training DataFrame shape: (2878, 32)

Target distribution:
requester_received_pizza
False    2163
True      715
Name: count, dtype: int64
Positive rate: 0.248

Missing values in key columns:
requester_received_pizza: 0 (0.0%)
request_text: 0 (0.0%)
request_title: 0 (0.0%)
requester_account_age_in_days_at_request: 0 (0.0%)
requester_user_flair: 2163 (75.2%)


In [4]:
# Feature engineering - basic numerical features
print("Creating basic features...")

# Numerical features that should be available at prediction time
numerical_features = [
    'requester_account_age_in_days_at_request',
    'requester_number_of_comments_at_request',
    'requester_number_of_posts_at_request',
    'requester_number_of_subreddits_at_request',
    'requester_upvotes_minus_downvotes_at_request',
    'requester_upvotes_plus_downvotes_at_request',
    'requester_number_of_comments_in_raop_at_request',
    'requester_number_of_posts_on_raop_at_request',
    'requester_days_since_first_post_on_raop_at_request'
]

# Create feature DataFrame
features_df = train_df[numerical_features].copy()

# Fill missing values with median
for col in numerical_features:
    if features_df[col].isnull().any():
        median_val = features_df[col].median()
        features_df[col].fillna(median_val, inplace=True)
        print(f"Filled missing values in {col} with median: {median_val:.2f}")

# Text-based features
print("\nCreating text features...")
features_df['title_length'] = train_df['request_title'].fillna('').str.len()
features_df['text_length'] = train_df['request_text'].fillna('').str.len()
features_df['title_word_count'] = train_df['request_title'].fillna('').str.split().str.len()
features_df['text_word_count'] = train_df['request_text'].fillna('').str.split().str.len()

# User flair feature
print("Creating user flair feature...")
features_df['has_flair'] = (train_df['requester_user_flair'].notna() & 
                           (train_df['requester_user_flair'] != 'None')).astype(int)

# Post was edited
features_df['post_was_edited'] = train_df['post_was_edited'].astype(int)

print(f"Final feature shape: {features_df.shape}")
print(f"Features: {list(features_df.columns)}")

Creating basic features...

Creating text features...
Creating user flair feature...
Final feature shape: (2878, 15)
Features: ['requester_account_age_in_days_at_request', 'requester_number_of_comments_at_request', 'requester_number_of_posts_at_request', 'requester_number_of_subreddits_at_request', 'requester_upvotes_minus_downvotes_at_request', 'requester_upvotes_plus_downvotes_at_request', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_posts_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_request', 'title_length', 'text_length', 'title_word_count', 'text_word_count', 'has_flair', 'post_was_edited']


In [5]:
# Prepare data for modeling
X = features_df.copy()
y = train_df['requester_received_pizza'].astype(int)

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Feature types:\n{X.dtypes.value_counts()}")

# Check for any remaining missing values
missing_summary = X.isnull().sum()
if missing_summary.sum() > 0:
    print("\nRemaining missing values:")
    print(missing_summary[missing_summary > 0])
else:
    print("\nNo missing values in features!")

X shape: (2878, 15), y shape: (2878,)
Feature types:
int64      13
float64     2
Name: count, dtype: int64

No missing values in features!


In [ ]:
# Load test data and create predictions
print("Loading test data...")
with open('/home/data/test.json', 'r') as f:
    test_data = json.load(f)

test_df = pd.DataFrame(test_data)
print(f"Test samples: {len(test_df)}")

# Create same features for test data
test_features = test_df[numerical_features].copy()

# Fill missing values with training median
for col in numerical_features:
    if test_features[col].isnull().any():
        median_val = features_df[col].median()  # Use training median
        test_features[col].fillna(median_val, inplace=True)

# Text features - use request_text_edit_aware
test_features['title_length'] = test_df['request_title'].fillna('').str.len()
test_features['text_length'] = test_df['request_text_edit_aware'].fillna('').str.len()
test_features['title_word_count'] = test_df['request_title'].fillna('').str.split().str.len()
test_features['text_word_count'] = test_df['request_text_edit_aware'].fillna('').str.split().str.len()

# User flair feature
test_features['has_flair'] = (test_df['requester_user_flair'].notna() & 
                             (test_df['requester_user_flair'] != 'None') &
                             (test_df['requester_user_flair'] != None)).astype(int)

# Post was edited
test_features['post_was_edited'] = test_df['post_was_edited'].astype(int)

print(f"Test features shape: {test_features.shape}")

# Train final model on full training data
print("\nTraining final model on full training data...")
train_data = lgb.Dataset(X, label=y)
final_model = lgb.train(
    params,
    train_data,
    num_boost_round=500,
    valid_sets=[train_data],
    callbacks=[lgb.log_evaluation(0)]
)

# Make predictions on test set
test_predictions = final_model.predict(test_features)
print(f"Test predictions shape: {test_predictions.shape}")
print(f"Prediction range: [{test_predictions.min():.4f}, {test_predictions.max():.4f}]")
print(f"Mean prediction: {test_predictions.mean():.4f}")

In [ ]:
# Load test data and create predictions
print("Loading test data...")
with open('/home/data/test.json', 'r') as f:
    test_data = json.load(f)

test_df = pd.DataFrame(test_data)
print(f"Test samples: {len(test_df)}")

# Create same features for test data
test_features = test_df[numerical_features].copy()

# Fill missing values with training median
for col in numerical_features:
    if test_features[col].isnull().any():
        median_val = features_df[col].median()  # Use training median
        test_features[col].fillna(median_val, inplace=True)

# Text features
test_features['title_length'] = test_df['request_title'].fillna('').str.len()
test_features['text_length'] = test_df['request_text'].fillna('').str.len()
test_features['title_word_count'] = test_df['request_title'].fillna('').str.split().str.len()
test_features['text_word_count'] = test_df['request_text'].fillna('').str.split().str.len()

# User flair feature
test_features['has_flair'] = (test_df['requester_user_flair'].notna() & 
                             (test_df['requester_user_flair'] != 'None')).astype(int)

# Post was edited
test_features['post_was_edited'] = test_df['post_was_edited'].astype(int)

print(f"Test features shape: {test_features.shape}")

# Train final model on full training data
print("\nTraining final model on full training data...")
train_data = lgb.Dataset(X, label=y)
final_model = lgb.train(
    params,
    train_data,
    num_boost_round=500,
    valid_sets=[train_data],
    callbacks=[lgb.log_evaluation(0)]
)

# Make predictions on test set
test_predictions = final_model.predict(test_features)
print(f"Test predictions shape: {test_predictions.shape}")
print(f"Prediction range: [{test_predictions.min():.4f}, {test_predictions.max():.4f}]")
print(f"Mean prediction: {test_predictions.mean():.4f}")

In [ ]:
# Create submission file
submission_df = pd.DataFrame({
    'request_id': test_df['request_id'],
    'requester_received_pizza': test_predictions
})

print("Submission preview:")
print(submission_df.head())
print(f"\nSubmission shape: {submission_df.shape}")

# Save submission
submission_path = '/home/submission/submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"\nSubmission saved to: {submission_path}")

# Verify submission format matches sample
sample_sub = pd.read_csv('/home/data/sampleSubmission.csv')
print(f"\nSample submission shape: {sample_sub.shape}")
print(f"Column match: {list(submission_df.columns) == list(sample_sub.columns)}")
print(f"ID match (first 5): {all(submission_df['request_id'].head() == sample_sub['request_id'].head())}")